In [25]:
import random
import math

with open("file-tsp.txt") as f:
    data = f.readlines()

cities = [(float(line.split()[0]), float(line.split()[1])) for line in data]

#parameters
POPULATION_SIZE = 50
ELITE_SIZE = 5
MUTATION_RATE = 0.01
GENERATIONS = 1500

def distance(city1, city2):
    x1, y1 = city1
    x2, y2 = city2
    return math.sqrt((x1-x2)**2 + (y1-y2)**2)

def tour_distance(tour):
    total_distance = 0
    for i in range(len(tour)):
        total_distance += distance(cities[tour[i]], cities[tour[(i+1)%len(tour)]])
    return total_distance

In [26]:
def generate_population(population_size):
    population = []
    for i in range(population_size):
        tour = list(range(len(cities)))
        random.shuffle(tour)
        population.append(tour)
    return population

def tournament_selection(population, k):
    parents = []
    for i in range(2):
        tournament = random.sample(population, k)
        parents.append(min(tournament, key=tour_distance))
    return parents

def order_crossover(parent1, parent2):
    child1 = [-1] * len(parent1)
    child2 = [-1] * len(parent2)
    cut1, cut2 = sorted(random.sample(range(len(parent1)), 2))
    for i in range(cut1, cut2):
        child1[i] = parent1[i]
        child2[i] = parent2[i]
    for i in range(len(parent1)):
        if parent2[i] not in child1:
            for j in range(len(parent1)):
                if child1[j] == -1:
                    child1[j] = parent2[i]
                    break
        if parent1[i] not in child2:
            for j in range(len(parent2)):
                if child2[j] == -1:
                    child2[j] = parent1[i]
                    break
    return child1, child2

In [27]:
def mutate(tour):
    if random.random() < MUTATION_RATE:
        i, j = random.sample(range(len(tour)), 2)
        tour[i], tour[j] = tour[j], tour[i]

def create_next_generation(population):
    elite = sorted(population, key=tour_distance)[:ELITE_SIZE]
    new_population = elite.copy()
    while len(new_population) < POPULATION_SIZE:
        parent1, parent2 = tournament_selection(population, 5)
        child1, child2 = order_crossover(parent1, parent2)
        mutate(child1)
        mutate(child2)
        new_population.append(child1)
        new_population.append(child2)
    return new_population

In [28]:
# main function
def genetic_algorithm():
    population = generate_population(POPULATION_SIZE)
    for i in range(GENERATIONS):
        population = create_next_generation(population)
        best_tour = min(population, key=tour_distance)
        print(f"Generation {i+1}: Best distance = {tour_distance(best_tour):.2f}")
    return best_tour

best_tour = genetic_algorithm()
print("Best tour found:")
for city in best_tour:
    print(f"City {city+1}: {cities[city]}")
print(f"Total distance of best tour: {tour_distance(best_tour):.2f}")

Generation 1: Best distance = 437.98
Generation 2: Best distance = 427.52
Generation 3: Best distance = 420.10
Generation 4: Best distance = 383.19
Generation 5: Best distance = 363.30
Generation 6: Best distance = 363.30
Generation 7: Best distance = 360.40
Generation 8: Best distance = 353.96
Generation 9: Best distance = 352.60
Generation 10: Best distance = 343.53
Generation 11: Best distance = 326.85
Generation 12: Best distance = 319.19
Generation 13: Best distance = 301.14
Generation 14: Best distance = 301.14
Generation 15: Best distance = 295.75
Generation 16: Best distance = 295.75
Generation 17: Best distance = 289.14
Generation 18: Best distance = 289.14
Generation 19: Best distance = 289.14
Generation 20: Best distance = 286.99
Generation 21: Best distance = 286.99
Generation 22: Best distance = 286.99
Generation 23: Best distance = 286.99
Generation 24: Best distance = 286.99
Generation 25: Best distance = 286.99
Generation 26: Best distance = 282.13
Generation 27: Best d

**MA's 2-opt algorithm **

In [31]:

MAX_EVALUATIONS = 15000

#2-opt local search
def local_search(tour):
    improved = True
    while improved:
        improved = False
        for i in range(len(tour)-2):
            for j in range(i+2, len(tour)):
                if distance(cities[tour[i]], cities[tour[j-1]]) + distance(cities[tour[i+1]], cities[tour[j]]) < \
                   distance(cities[tour[i]], cities[tour[i+1]]) + distance(cities[tour[j-1]], cities[tour[j]]):
                    tour[i+1:j] = reversed(tour[i+1:j])
                    improved = True
    return tour

def create_candidate(population):
    parent1 = random.choice(population)
    parent2 = random.choice(population)
    child, _ = order_crossover(parent1, parent2)
    mutate(child)
    return child

def select_replacement(population, candidate):
    individuals = random.sample(population, 2)
    if tour_distance(candidate) < tour_distance(individuals[0]):
        return individuals[0], candidate
    elif tour_distance(candidate) < tour_distance(individuals[1]):
        return individuals[1], candidate
    else:
        return None, None

#Main function
def memetic_algorithm():
    population = generate_population(POPULATION_SIZE)
    for i in range(len(population)):
        population[i] = local_search(population[i])
    #Evaluate fitness
    evaluations = POPULATION_SIZE
    fitness_values = [tour_distance(tour) for tour in population]
    while evaluations < MAX_EVALUATIONS:
        candidate = create_candidate(population)

        candidate = local_search(candidate)

        candidate_fitness = tour_distance(candidate)
        evaluations += 1

        replace_indv, replace_candidate = select_replacement(population, candidate)

        if replace_indv:
            population[population.index(replace_indv)] = replace_candidate

        fitness_values = [tour_distance(tour) for tour in population]

    best_tour = population[fitness_values.index(min(fitness_values))]
    best_distance = min(fitness_values)

    print("Best tour found: ", best_tour)
    print("Distance of best tour: ", best_distance)
memetic_algorithm()

Best tour found:  [26, 34, 37, 36, 42, 47, 48, 49, 44, 41, 38, 40, 43, 46, 45, 39, 35, 31, 28, 27, 19, 12, 7, 13, 16, 15, 18, 22, 21, 29, 25, 24, 32, 33, 30, 23, 20, 17, 11, 5, 2, 10, 9, 1, 3, 0, 4, 6, 8, 14]
Distance of best tour:  119.94240011114012


file-tsp : (GA,MA) (212.16,119.94),(204.28, 119.942),(198.57,119.94)